In [3]:
import kernel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import xgboost as xgb
import numpy as np
import networkx as nx
from sktime.utils import plot_series
from nonconformist.cp import IcpRegressor
from nonconformist.nc import RegressorNc, AbsErrorErrFunc
from matplotlib import pyplot as plt
import lingam

target_countries = ['Senegal', 'Mauritania', 'Mali', 'Burkina Faso', 'Niger', 'Nigeria', 'Cameroon', 'Chad', 'Central African Republic', 'South Sudan', 'Sudan', 'Eritrea']

# dataset = pd.read_csv('datasets/DatasetMonthly.csv').rename(columns={'COUNTRY': 'country', 'MY': 'my', 'FATALITIES': 'Fatalities', 'Political Stability and Absence of Violence/Terrorism: Estimate' :'Political Stability & Absence of Violence'})
dataset = pd.read_excel('datasets/Dataset_Monthly_World.xlsx')

window_length = 5

forecasting_horizon = 1
target = 'Fatalities'

dataset[target] = dataset[target].astype(float)
for country in np.unique(dataset['country']):
    for lag in range(1, window_length+1):
        dataset.loc[dataset['country']==country, f'{target}_lag{lag}'] = dataset.loc[dataset['country']==country, target].shift(lag).fillna(0)
    dataset.loc[dataset['country']==country, target] = dataset.loc[dataset['country']==country, target].shift(1-forecasting_horizon)
    
dataset = dataset.dropna()
print(dataset.info())


<class 'pandas.core.frame.DataFrame'>
Index: 16092 entries, 0 to 20046
Data columns (total 22 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   country                                    16092 non-null  object        
 1   date                                       16092 non-null  datetime64[ns]
 2   Fatalities                                 16092 non-null  float64       
 3   Demonstrations                             16092 non-null  int64         
 4   Political violence                         16092 non-null  int64         
 5   Strategic developments                     16092 non-null  int64         
 6   Control of Corruption                      16092 non-null  float64       
 7   Government Effectiveness                   16092 non-null  float64       
 8   Political Stability & Absence of Violence  16092 non-null  float64       
 9   Regulatory Quality    

In [4]:

mask = dataset[f'{target}'] > 50
# mask = dataset['Political Stability and Absence of Violence']<-1
# mask = dataset['target']/dataset['Population']>1
d2 = dataset[mask]
d1 = dataset[~mask]

results = []
results_original = []

if 'my' in dataset.columns:
    time_col = 'my'
if 'date' in dataset.columns:
    time_col = 'date'

country = 'Burkina Faso'

data = dataset.loc[dataset['country'] == country].copy()
data = data.sort_values(by=time_col)
data = data.drop('country', axis = 1)
data = data.ffill().bfill()
if 'MY' in data.columns:
    dates = data['MY']
    data = data.drop('MY', axis=1)
if 'EVENT_DATE' in data.columns:
    dates = data['EVENT_DATE']
    data = data.drop('EVENT_DATE', axis=1)

data = data.reset_index(drop=True)

train, test = train_test_split(data, test_size=0.40, shuffle=False)
cal, test = train_test_split(test, test_size=0.50, shuffle=False)

causal_model = lingam.DirectLiNGAM()
causal_model.fit(dataset[~dataset.isin(test)].dropna().drop('country', axis=1).drop(time_col, axis=1).copy())

causal_graph = nx.DiGraph(causal_model.adjacency_matrix_)

data_aug = kernel.causal_data_augmentation(train, d2[~d2.isin(test)].dropna().drop('country', axis=1).drop(time_col, axis=1).copy().sample(500), causal_graph, target)
data_aug_2 = kernel.causal_data_augmentation(train, d1[~d1.isin(test)].dropna().drop('country', axis=1).drop(time_col, axis=1).copy().sample(500), causal_graph, target)

fatalities_aug = data_aug[target]
fatalities_aug_2 = data_aug_2[target]

fatalities_aug[fatalities_aug < 0] = 0
fatalities_aug_2[fatalities_aug_2 < 0] = 0

data_aug = pd.concat([train, data_aug, data_aug_2], axis=0)

for column in data_aug.columns:
    if target in column:
        data_aug[column] = np.where(data_aug[column]<0, np.zeros_like(data_aug[column]), data_aug[column])

xtrain_original = train.drop(target, axis=1)
y_train_original = train[target]

xtest = test.drop(target, axis=1)
ytest = test[target]



cal_aug = kernel.causal_data_augmentation(cal, d2[~d2.isin(test)].dropna().drop('country', axis=1).drop(time_col, axis=1).copy().sample(200), causal_graph, target)
cal_aug_2 = kernel.causal_data_augmentation(cal, d1[~d1.isin(test)].dropna().drop('country', axis=1).drop(time_col, axis=1).copy().sample(200), causal_graph, target)

cal = pd.concat([cal, cal_aug, cal_aug_2], axis=0)

xcal = cal.drop(target, axis=1)
ycal = cal[target]

xtrain = data_aug.drop(target, axis=1)
ytrain = data_aug[target]

plot_series(fatalities_aug, fatalities_aug_2, y_train_original, labels=["Augmented series","Augmented series 2","Original series"], title='Fatalities augmentation, ' + country)

forecaster = xgb.XGBRegressor(n_estimators=10000)
forecaster.fit(xtrain, ytrain)

forecaster_original = xgb.XGBRegressor(n_estimators=10000)
forecaster_original.fit(xtrain_original, y_train_original)

# Make forecasts for the test set
y_pred = forecaster.predict(xtest)
y_pred[y_pred < 0] = 0
y_pred_original = forecaster_original.predict(xtest)
y_pred_original[y_pred_original < 0] = 0

# Plot the actual vs. predicted delays
y_pred = pd.DataFrame(y_pred, index=ytest.index)
y_pred_original = pd.DataFrame(y_pred_original, index=ytest.index)
fig, ax = plot_series(y_pred_original, pd.DataFrame(ytest), pd.DataFrame(y_pred), labels=["Forecast only real data","True values", "Forecast with synthetic data"], title='Fatalities forecast, ' + country)
fig.savefig('results_burkina_fatalities.eps')
results = mean_absolute_error(ytest, y_pred), mean_absolute_percentage_error(ytest, y_pred)
results_original = mean_absolute_error(ytest, y_pred_original), mean_absolute_percentage_error(ytest, y_pred_original)

nc = RegressorNc(forecaster, AbsErrorErrFunc())
icp = IcpRegressor(nc)

# Calibrate the conformal predictor
icp.calibrate(xcal.values, ycal.values)

# Make predictions
predictions = icp.predict(xtest.values, significance=0.05)
preds_below = np.where(predictions[:, 0]<0, np.zeros_like(predictions[:, 0]), predictions[:, 0])

# Plot the prediction intervals
plt.figure(figsize=(10, 5))
plt.plot(ytest.index, ytest, 'o', color='black', label='True values')
plt.plot(ytest.index, preds_below, 'r--', label='Lower bound')
plt.plot(ytest.index, predictions[:, 1], 'b--', label='Upper bound')
plt.ylabel(target)
plt.xlabel('MONTHS')
plt.fill_between(ytest.index, preds_below, predictions[:, 1], color='red', alpha=0.2)
plt.title('Calibration of the forecaster, 95\% confidence')
plt.legend()
plt.savefig('calibration_burkina_fatalities.eps')
plt.show()

nc = RegressorNc(forecaster_original, AbsErrorErrFunc())
icp = IcpRegressor(nc)

# Calibrate the conformal predictor
icp.calibrate(xcal.values, ycal.values)

# Make predictions
predictions = icp.predict(xtest.values, significance=0.05)

# Plot the prediction intervals
plt.figure(figsize=(10, 5))
preds_below = np.where(predictions[:, 0]<0, np.zeros_like(predictions[:, 0]), predictions[:, 0])
plt.plot(ytest.index, ytest, 'o', color='black', label='True values')
plt.plot(ytest.index, preds_below, 'r--', label='Lower bound')
plt.plot(ytest.index, predictions[:, 1], 'b--', label='Upper bound')
plt.ylabel(target)
plt.xlabel('MONTHS')
plt.fill_between(ytest.index, preds_below, predictions[:, 1], color='red', alpha=0.2)
plt.title('Calibration of the original forecaster')
plt.legend()
plt.show()


print(results)
print(results_original)

<>:112: SyntaxWarning: invalid escape sequence '\%'
<>:112: SyntaxWarning: invalid escape sequence '\%'
C:\Users\alber\AppData\Local\Temp\ipykernel_8612\3704038426.py:112: SyntaxWarning: invalid escape sequence '\%'
  plt.title('Calibration of the forecaster, 95\% confidence')
0it [00:00, ?it/s]


TypeError: Type of new_samples is [[0.09801563620567325]
 [0.1004412323236466]
 [0.1055349841713906]
 [0.1103861764073373]
 [0.1149948090314866]
 [0.1192696822807194]
 [0.1232107961550356]
 [0.1268181506544353]
 [0.1300917457789184]
 [0.1331227812916042]
 [0.1169250978777808]
 [0.09702320086459335]
 [0.07421058850983793]
 [0.05502521805465234]
 [0.03583984759946675]
 [0.01665447714428116]
 [-0.01958013139665117]
 [-0.05681793733189502]
 [-0.09396454350401967]
 [-0.1310199499130251]
 [-0.1696074840923154]
 [-0.2053853782514735]
 [-0.4500110248724617]
 [-0.5048809535801413]
 [-0.5182110344370209]
 [-0.4946907435854278]
 [-0.515637241303921]
 [-0.5365837390224142]
 [-0.7450718556841216]
 [-0.7489691153168679]
 [-0.7546801418066026]
 [-0.7603911682963372]
 [-0.7839587976535168]
 [-0.7691788251201319]
 [-0.7749817669391639]
 [-0.7807847087581958]
 [-0.7865876505772279]
 [-0.8259327734510103]
 [-0.8390251696109776]
 [-0.8494886209567392]
 [-0.8599520723025008]
 [-0.8704155236482624]
 [-0.900

In [ ]:
nc = RegressorNc(forecaster, AbsErrorErrFunc())
icp = IcpRegressor(nc)

# Calibrate the conformal predictor
icp.calibrate(xcal.values, ycal.values)

# Make predictions
predictions = icp.predict(xtest.values, significance=0.05)
preds_below = np.where(predictions[:, 0]<0, np.zeros_like(predictions[:, 0]), predictions[:, 0])

# Plot the prediction intervals
plt.figure(figsize=(10, 5))
preds_below = np.where(predictions[:, 0]<0, np.zeros_like(predictions[:, 0]), predictions[:, 0])
plt.plot(ytest.index, ytest, 'o', color='black', label='True values')
plt.plot(ytest.index, preds_below, 'r--', label='Lower bound')
plt.plot(ytest.index, predictions[:, 1], 'b--', label='Upper bound')
plt.ylabel(target)
plt.xlabel('MONTHS')
plt.fill_between(ytest.index, preds_below, predictions[:, 1], color='red', alpha=0.2)
plt.title('Calibration of the forecaster with augmented data, 95% confidence, ' + country)
plt.legend()
plt.savefig('calibration_burkina_fatalities.eps')
plt.show()